## Backpropagation (오차역전파)

## 5.4.1 곱셈 계층

In [2]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
    
        return out

    def backward(self, dout):
        dx = dout * self.y # x와y를 바꾼다.
        dy = dout * self.x 

        return dx, dy
    

In [3]:
apple = 100
apple_num = 2
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(int(price))

220


## 5.4.2 덧셈 계층

In [4]:
class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y
        return out

    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

In [24]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()


# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

print(int(price))


# 역전파
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
print("dall_price -> ", dall_price)
print("dtax -> ", dtax)

dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
print("dapple_price -> ", dapple_price)
print("dorange_price -> ", dorange_price)

dorange, dorange_num = mul_orange_layer.backward(dorange_price)
print("dorange -> ", int(dorange))
print("dorange_num -> ", int(dorange_num))

dapple, dapple_num = mul_apple_layer.backward(dapple_price)
print("dapple -> ", dapple)
print("dapple_num -> ", int(dapple_num))

715
dall_price ->  1.1
dtax ->  650
dapple_price ->  1.1
dorange_price ->  1.1
dorange ->  3
dorange_num ->  165
dapple ->  2.2
dapple_num ->  110


## 5.5 활성화 함수 계층 구현하기

In [25]:
class Relu:

    def __init__(self):
        # T/F로 구성된 넘파이 배열
        # 순전파 입력인 x의 원소 값이 0 이하인 인덱스는 True.
        # 그 외 (0보다 큰 원소)는 False.
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx



In [30]:
import numpy as np

x = np.array( [[1.0, -0.5], [-2.0, 3.0]])
print(x)

mask = ( x <= 0 )
mask

[[ 1.  -0.5]
 [-2.   3. ]]


array([[False,  True],
       [ True, False]])

## 5.5.2 Sigmoid 계층

In [31]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out

        return out

    def backward(self, dout):
        # dσ(x)/dx = σ(x)⋅(1−σ(x))
        dx = dout * (1.0 - self.out) * self.out
        return dx

## 5.6 Affine/Softmax 계층 구현하기

In [33]:
X_dot_W = np.array([[0,0,0], [10,10,10]])
B = np.array([1,2,3])

print(X_dot_W)
print(X_dot_W + B)

[[ 0  0  0]
 [10 10 10]]
[[ 1  2  3]
 [11 12 13]]


In [34]:
dY = np.array([[1,2,3], [4,5,6]])
dY

array([[1, 2, 3],
       [4, 5, 6]])

In [36]:
# axis=0은 열 방향으로 합을 계산
dB = np.sum(dY, axis=0)
dB

array([5, 7, 9])

In [37]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None

    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b

        return out

    # dout => 이전 레이어로부터 전달된 그래디언트(기울기)
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
    
        return dx


## 5.6.3 Softmax - with - Loss 계층

Softmax 계층은 입력 (a1,a2,a3)를 정규화하여 (y1,y2,y3)를 출력  <br>
Cross Entropy Error 계층은 Softmax의 출력 (y1,y2,y3)와 정답 레이블 (t1,t2,t3)를 받고, 이 데이터들로부터 손실 L을 출력

예를들어, 정답 레이블이 (0,1,0) 일 때 SoftMax 계층이 (0.3, 0.2, 0.5)를 출력 <br>
정답 레이블의 인덱스는 1 <br>
그런데 출력에서는 이때의 확률이 0.2에 불과 <br>
Softmax 계층의 역전파는 (0.3, -0.8, 0.5)라는 큰 오차를 전파 <br>  --> (0.3-0, 0.2-1, 0.5-0) <br>
결과적으로, Softmax 계층의 앞 계층들은 그 큰 오차로부터 더 크게 움직임 (학습 정도가 커짐)


In [38]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None #손실
        self.y = None #softmax 출력
        self.t = None #정답 레이블 (ont-hot encoding)

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        # 역전파 때는 전파하는 값을 배치의 수로 나눠 데이터 "1개당" 오차를 앞 계층으로 전파.
        dx = (self.y - self.t) / batch_size
        return dx